
<br>
Example of running GROMACS with lithops<br>


# GROMACS Computations

GROMACS is a versatile package to perform molecular dynamics, i.e. simulate the Newtonian equations of motion for systems with hundreds to millions of particles.

In this notebook, GROMACS software is run with parameters over cloud functions via shell command. We demonstrate how complex computations and softwares like this can be handled using Lithops and Cloud resources.

## Installing the dependencies

Install the dependencies within an execution environment with the attached [requirements.txt](./requirements.txt) file.

In [ ]:
!pip install -r requirements.txt

In [ ]:
RUNTIME_NAME = "lithops-gromacs-runtime:1"

Build a runtime with the necessary dependencies. You have to substitute aws_lambda with your desired computation backend, if necessary.

**NOTE: building GROMACS runtime takes significant time, could be more than an hour.**

In [ ]:
!lithops runtime build -b aws_lambda -f runtime/Dockerfile.py3.10 $RUNTIME_NAME --debug

In [ ]:
import lithops
import os
import zipfile
import time


## Execution

Here in this notebook there is only one function that we execute over cloud functions. This function downloads the benchMEM benchmark set and executes the shell command for running the gromacs software with the given parameters and uploads the results to the specified bucket.

You should replace BUCKET with your bucket name in which you put the dataset.

In [ ]:
BUCKET = "BUCKET"
KEY = "benchMEM.zip"

In [ ]:
temp_dir = '/tmp'
iterdata = [1]

def sh_cmd_executor(x, param1, storage):

    print (param1)
    filename = 'benchMEM.zip'
    outfile = os.path.join(temp_dir, filename)
    if not os.path.isfile(filename):
        storage.download_file(BUCKET, KEY, file_name = outfile)
        with zipfile.ZipFile(outfile, 'r') as zip_ref:
            print('Extracting file to %s' % temp_dir)
            zip_ref.extractall(temp_dir)
    else:
        print(filename, " already exists")
    os.chdir(temp_dir)
    cmd = "/usr/local/gromacs/bin/gmx mdrun -nt 4 -s benchMEM.tpr -nsteps 1000 -resethway"
    st = time.time()
    import subprocess
    subprocess.call(cmd, shell=True)
    run_time = time.time() - st

    # upload results to IBM COS
    res = ['confout.gro', 'ener.edr', 'md.log',  'state.cpt']
    for name in res:
        f = open(os.path.join(temp_dir, name), "rb")
        storage.put_object(bucket=BUCKET, key=os.path.join('gmx-mem', name), body=f)
    with open('md.log', 'r') as file:
        data = file.read()
    return {'run_time': run_time, 'md_log': data}

We use lithops and the pre-built runtime to run gromacs with the parameters given. **Currently, the runtime cactusone/lithops-gromacs:1.0.2 uses Python3.8, so you must run the application with Python3.8.** And we get the results from GROMACS at the end of the process.

In [ ]:
if __name__ == '__main__':
    # Example of using bechMEM from https://www.mpibpc.mpg.de/grubmueller/bench
    param1 = 'param1 example'
    total_start = time.time()
    fexec = lithops.FunctionExecutor(runtime=RUNTIME_NAME, runtime_memory=2048, log_level='DEBUG')
    fexec.map(sh_cmd_executor, iterdata, extra_args=(param1,))
    res = fexec.get_result()
    fexec.clean()
    print ("GROMACS execution time {}".format(res[0]['run_time']))
    print ("Total execution time {}".format(time.time()-total_start))
    print (res[0]['md_log'])